In [1]:
# Import modules and packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
from datetime import datetime

from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard

%matplotlib inline

Using TensorFlow backend.


In [3]:
dataset_train = pd.read_csv('/Users/dennis/git/Data/pump_data.csv', sep=';', low_memory=False , decimal=',')

In [4]:
dataset_train.head()

,Unnamed: 0,Date,03:I3SROE,03:T3362,03:PI3061A,03:PI3061B,03:PI3061C,03:PD3066,03:T3492,03:T3358,...,03:E3851,03:T3632,03:T3633,03:T3634,03:FC3195,03:FV3195,03:FC3197,03:FV3197,03:FC3199,03:FV3199
0,0,01.01.19 00:00,"890,187","77,923","3,025","3,043","3,055",No Data,"40,456","97,602",...,"0,156","16,62","16,661","13,997","358,322",100,"362,52",100,"392,51",100
1,1,01.01.19 00:10,"890,262","77,893","3,024","3,043","3,053",No Data,"40,415","97,178",...,"0,157","16,675","16,676","13,985","358,897",100,"362,505",100,"393,009",100
2,2,01.01.19 00:20,"890,337","77,863","3,024","3,043","3,051",No Data,"40,375","97,215",...,"0,158","16,711","16,726","14,02","358,984",100,"362,305",100,"392,767",100
3,3,01.01.19 00:30,"890,294","77,833","3,023","3,043","3,051",No Data,"40,41","97,602",...,"0,159","16,716","16,784","14,035","362,205",100,"361,171",100,"391,512",100
4,4,01.01.19 00:40,"890,239","77,823","3,023","3,043","3,052",No Data,"40,41","97,438",...,"0,16","16,75","16,815","14,065","361,954",100,"360,852",100,"390,736",100


In [5]:
dataset_train = dataset_train.replace(to_replace='N_LAUF', value='0')
dataset_train = dataset_train.replace(to_replace='LAUF', value='1')
dataset_train = dataset_train.replace(to_replace='No Data', value='-1001')
dataset_train = dataset_train.replace(to_replace='Over Range', value='-1002')
dataset_train = dataset_train.replace(to_replace='I/O Timeout', value='-1003')
dataset_train = dataset_train.replace(to_replace='Calc Failed', value='-1004')
dataset_train = dataset_train.replace(to_replace='Error', value='-1005')
dataset_train = dataset_train.replace(to_replace='Configure', value='-1006')
dataset_train = dataset_train.replace(to_replace='Scan Off', value='-1007')
dataset_train = dataset_train.replace(to_replace='Comm Fail', value='-1008')
dataset_train = dataset_train.replace(to_replace='Shutdown', value='-1009')
dataset_train = dataset_train.replace(to_replace='NaN', value='-1010')
dataset_train = dataset_train.replace(to_replace='Bad Input', value='-1011')
dataset_train = dataset_train.replace(to_replace='Bad', value='-1012')

In [6]:
dataset_train = dataset_train.drop(['Unnamed: 0'], axis = 1)
dataset_train = dataset_train.drop(['03:FV3195'], axis = 1)
dataset_train = dataset_train.drop(['03:FV3197'], axis = 1)
dataset_train = dataset_train.drop(['03:FV3199'], axis = 1)

In [7]:
dataset_train = dataset_train[1:].replace(',','.', regex=True)

In [8]:
dataset_train.dtypes

Date            object
03:I3SROE       object
03:T3362        object
03:PI3061A      object
03:PI3061B      object
03:PI3061C      object
03:PD3066       object
03:T3492        object
03:T3358        object
03:TC3550       object
03:TC3550A      object
03:FV3762       object
03:FC3762       object
03:TC3730       object
03:TV3730A      object
03:TV3730B      object
03:TV3730C      object
03:T3731        object
03:P3649        object
03:P3065        object
03:LC3661       object
03:LI3661A      object
03:LI3661B      object
03:LI3661C      object
03:YV4978       object
03:FC3075.OP    object
03:FC3075       object
03:T3361        object
03:T3359        object
03:T3360        object
03:N3722_1      object
03:N3721_1      object
03:N3725_1      object
03:E3849        object
03:T3626        object
03:T3627        object
03:T3628        object
03:N3726_1      object
03:E3850        object
03:T3629        object
03:T3630        object
03:T3631        object
03:N3723_1      object
03:N3727_1 

In [9]:
for column in dataset_train[1:]:
    dataset_train[column] = pd.to_numeric(dataset_train[column], errors='coerce')

In [10]:
dataset_train.dtypes

Date            float64
03:I3SROE       float64
03:T3362        float64
03:PI3061A      float64
03:PI3061B      float64
03:PI3061C      float64
03:PD3066       float64
03:T3492        float64
03:T3358        float64
03:TC3550       float64
03:TC3550A      float64
03:FV3762       float64
03:FC3762       float64
03:TC3730       float64
03:TV3730A      float64
03:TV3730B      float64
03:TV3730C      float64
03:T3731        float64
03:P3649        float64
03:P3065        float64
03:LC3661       float64
03:LI3661A      float64
03:LI3661B      float64
03:LI3661C      float64
03:YV4978       float64
03:FC3075.OP    float64
03:FC3075       float64
03:T3361        float64
03:T3359        float64
03:T3360        float64
03:N3722_1      float64
03:N3721_1      float64
03:N3725_1      float64
03:E3849        float64
03:T3626        float64
03:T3627        float64
03:T3628        float64
03:N3726_1      float64
03:E3850        float64
03:T3629        float64
03:T3630        float64
03:T3631        

In [11]:
dataset_train['Durchsatz'] = dataset_train['03:FC3195'] + dataset_train['03:FC3197'] + dataset_train['03:FC3199']
dataset_train['Durchsatz']

1         1114.411
2         1114.056
3         1114.888
4         1113.542
5         1113.840
            ...   
574000     932.979
574001     933.390
574002     933.553
574003     932.625
574004     944.560
Name: Durchsatz, Length: 574004, dtype: float64

In [12]:
# Select features (columns) to be involved intro training and predictions
cols = list(dataset_train)[1:]

cols

['03:I3SROE',
 '03:T3362',
 '03:PI3061A',
 '03:PI3061B',
 '03:PI3061C',
 '03:PD3066',
 '03:T3492',
 '03:T3358',
 '03:TC3550',
 '03:TC3550A',
 '03:FV3762',
 '03:FC3762',
 '03:TC3730',
 '03:TV3730A',
 '03:TV3730B',
 '03:TV3730C',
 '03:T3731',
 '03:P3649',
 '03:P3065',
 '03:LC3661',
 '03:LI3661A',
 '03:LI3661B',
 '03:LI3661C',
 '03:YV4978',
 '03:FC3075.OP',
 '03:FC3075',
 '03:T3361',
 '03:T3359',
 '03:T3360',
 '03:N3722_1',
 '03:N3721_1',
 '03:N3725_1',
 '03:E3849',
 '03:T3626',
 '03:T3627',
 '03:T3628',
 '03:N3726_1',
 '03:E3850',
 '03:T3629',
 '03:T3630',
 '03:T3631',
 '03:N3723_1',
 '03:N3727_1',
 '03:E3851',
 '03:T3632',
 '03:T3633',
 '03:T3634',
 '03:FC3195',
 '03:FC3197',
 '03:FC3199',
 'Durchsatz']

In [93]:
dataset_train.dtypes

Date            float64
03:I3SROE       float64
03:T3362        float64
03:PI3061A      float64
03:PI3061B      float64
03:PI3061C      float64
03:PD3066       float64
03:T3492        float64
03:T3358        float64
03:TC3550       float64
03:TC3550A      float64
03:FV3762       float64
03:FC3762       float64
03:TC3730       float64
03:TV3730A      float64
03:TV3730B      float64
03:TV3730C      float64
03:T3731        float64
03:P3649        float64
03:P3065        float64
03:LC3661       float64
03:LI3661A      float64
03:LI3661B      float64
03:LI3661C      float64
03:YV4978       float64
03:FC3075.OP    float64
03:FC3075       float64
03:T3361        float64
03:T3359        float64
03:T3360        float64
03:N3722_1      float64
03:N3721_1      float64
03:N3725_1      float64
03:E3849        float64
03:T3626        float64
03:T3627        float64
03:T3628        float64
03:N3726_1      float64
03:E3850        float64
03:T3629        float64
03:T3630        float64
03:T3631        

In [13]:
training_set = dataset_train[cols].values

In [19]:
training_set

array([[ 890.262,   77.893,    3.024, ...,  362.505,  393.009, 1114.411],
       [ 890.337,   77.863,    3.024, ...,  362.305,  392.767, 1114.056],
       [ 890.294,   77.833,    3.023, ...,  361.171,  391.512, 1114.888],
       ...,
       [ 750.08 ,   73.618,    2.696, ...,  300.264,  333.121,  933.553],
       [ 751.267,   73.667,    2.699, ...,  299.913,  332.373,  932.625],
       [ 755.64 ,   73.563,    2.701, ...,  305.107,  338.502,  944.56 ]])

In [23]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
training_set_scaled = sc.fit_transform(training_set)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/extmath.py:765: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/extmath.py:706: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = op(x, *args, **kwargs)


In [24]:
training_set_scaled

array([[ 0.15041391,  0.03238712,  0.08155985, ...,  0.24313427,
         0.23257248,  0.17721778],
       [ 0.15087862,  0.03106443,  0.08155985, ...,  0.23974709,
         0.23034998,  0.17601331],
       [ 0.15061219,  0.02974175,  0.0815471 , ...,  0.22054178,
         0.21882418,  0.17883617],
       ...,
       [-0.71817797, -0.15609514,  0.07737748, ..., -0.81097315,
        -0.31743323, -0.43640635],
       [-0.71082311, -0.15393476,  0.07741573, ..., -0.81691765,
        -0.32430279, -0.43955491],
       [-0.68372725, -0.15852006,  0.07744124, ..., -0.72895258,
        -0.26801464, -0.39906125]])

In [30]:
trainX = training_set_scaled[:,1:50]
trainX

array([[ 0.03238712,  0.08155985,  0.08168098, ...,  0.08276647,
         0.24313427,  0.23257248],
       [ 0.03106443,  0.08155985,  0.08168098, ...,  0.08334011,
         0.23974709,  0.23034998],
       [ 0.02974175,  0.0815471 ,  0.08168098, ...,  0.10457793,
         0.22054178,  0.21882418],
       ...,
       [-0.15609514,  0.07737748,  0.07726913, ..., -0.30446599,
        -0.81097315, -0.31743323],
       [-0.15393476,  0.07741573,  0.07728189, ..., -0.30333849,
        -0.81691765, -0.32430279],
       [-0.15852006,  0.07744124,  0.07729464, ..., -0.29930324,
        -0.72895258, -0.26801464]])

In [31]:
trainY = training_set_scaled[:,50:51]
trainY

array([[ 0.17721778],
       [ 0.17601331],
       [ 0.17883617],
       ...,
       [-0.43640635],
       [-0.43955491],
       [-0.39906125]])

In [32]:
trainX.shape

(574004, 49)

In [33]:
trainY.shape

(574004, 1)